## Test time inference using trained model for net3D-threenets_pred_xyz

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset as dset
from torch.utils.data import DataLoader
from scipy.io import loadmat
from PIL import Image
import matplotlib.pyplot as plt
from logger import Logger
import os as os
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
torch.backends.cudnn.deterministic=True

plt.rcParams.update({'font.size': 15,'lines.markersize':10})


In [ ]:
from utils_single_point import *
from visualize import *
np.random.seed(190)
torch.manual_seed(120) ##set the random seed so when construct the 
#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_10_24 data')
os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_03 data')
#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_21 CCD XYZ sweep')
os.getcwd()

In [ ]:

lr_x,lr_y,lr_z=1e-2,1e-2,1e-2
bs_train=50
bs_test=200
num_epoch=2000
save_name='Depthnet4_4_3(128_128_32features)_adam_lr_x_'+str(lr_x)+'_lr_y_'+str(lr_y)+'_lr_z_'+str(lr_z)+'_bs_train_'+str(bs_train)+'_bs_test_'+str(bs_test)+'_numepoch_'+str(num_epoch)

"""
dataidx:specify what subset of the fulldata is used to split into train/test, 
e.g, for a particular fixed xy and sweep z,np.arange(0,1999,100)
for a particular fixed z and sweep xy, np.arange(100,200,1)
test+idx specify among the subset of the fulldata, which point is used as the testset (start from 1, ie, test_idx=1 means FS_all[0] )
"""
#data_idx=np.arange(0,396,36)## fixed  particular xy, sweep z 
#dataidx=np.arange(0,6) ## fixed  particular y，z, sweep x
data_idx=np.arange(0,1331) ##all data used for 3D regression
test_idx=(np.random.permutation(np.arange(1331))+1)[:bs_test] ##+1 since test_idx start from 1

In [ ]:

Full_dataset=FSdataset('18_11_03sweepxyz.mat',set='Full',data_idx=data_idx,test_idx=test_idx)

train_dataset=FSdataset('18_11_03sweepxyz.mat',set='Train',data_idx=data_idx,test_idx=test_idx)
train_loader=DataLoader(train_dataset, batch_size=bs_train,shuffle=True)
test_dataset=FSdataset('18_11_03sweepxyz.mat',set='Test',data_idx=data_idx,test_idx=test_idx)
test_loader=DataLoader(test_dataset, batch_size=bs_test,shuffle=False)

In [ ]:
#dtype = torch.FloatTensor ##run on cpu
dtype = torch.cuda.FloatTensor 

net_x=depthnet4(input_dim=4*4*2,num_features=128)
net_y=depthnet4(input_dim=4*4*2,num_features=128)
net_z=depthnet3(input_dim=4*4*2,num_features=32)

net_x.type(dtype)
net_y.type(dtype)
net_z.type(dtype)

net_x.load_state_dict(torch.load('logs_paper_data/'+save_name+'/model_x.pth'))
net_y.load_state_dict(torch.load('logs_paper_data/'+save_name+'/model_y.pth'))
net_z.load_state_dict(torch.load('logs_paper_data/'+save_name+'/model_z.pth'))

In [ ]:

#this part of code is added , previsouly the test loss is not the final model test loss since np.mod(epoch,logfreq)==0 is false for last epoch 
net_x.eval()
net_y.eval()
net_z.eval()
data=next(iter(test_loader))
FS_test_bat,trueXobj_test_bat,trueYobj_test_bat,trueZobj_test_bat=data['FS'],data['xyz'][:,0],data['xyz'][:,1],data['xyz'][:,2]
estXobj_test,estYobj_test,estZobj_test=net_x(Variable(FS_test_bat.type(dtype))),net_y(Variable(FS_test_bat.type(dtype))),net_z(Variable(FS_test_bat.type(dtype)))


test_bat=next(iter(test_loader))
FS_bat,trueXYZobj_bat=test_bat['FS'],test_bat['xyz'][:,:]
predXYZobj_bat=torch.cat((net_x(Variable(FS_bat.type(dtype))),net_y(Variable(FS_bat.type(dtype))),net_z(Variable(FS_bat.type(dtype)))),dim=1).data.cpu()
print(predXYZobj_bat)
print(trueXYZobj_bat)


show3Dpred(trueXYZobj_bat,predXYZobj_bat,howmany=10,save_name='logs_paper_data/'+save_name+'/3D_new')
show2D(trueXYZobj_bat,predXYZobj_bat,howmany=10,save_name='logs_paper_data/'+save_name+'/2Ds_new')

